# Create Fact Table

In [0]:
df_silver = spark.sql("select * from parquet.`abfss://silver@sacarsdatalake.dfs.core.windows.net/carsales`")

In [0]:
df_silver.display()

### Reading all the DIMS

In [0]:
df_dealer = spark.sql("SELECT * FROM cars_catalog.gold.dim_dealer")

df_model = spark.sql("SELECT * FROM cars_catalog.gold.dim_model")

df_branch = spark.sql("SELECT * FROM cars_catalog.gold.dim_branch")

df_date = spark.sql("SELECT * FROM cars_catalog.gold.dim_date")

### Bringing Keys to the Fact Table

In [0]:
df_fact = df_silver.join(df_branch, df_silver.Branch_ID == df_branch.Branch_ID, 'left')\
                   .join(df_dealer, df_silver.Dealer_ID == df_dealer.Dealer_ID, 'left')\
                   .join(df_model, df_silver.Model_ID == df_model.Model_ID, 'left')\
                   .join(df_date, df_silver.Date_ID == df_date.Date_ID, 'left')\
                   .select(df_silver.Revenue, df_silver.Units_Sold, df_silver.RevenuePerUnit, df_branch.dim_branch_key, df_dealer.dim_dealer_key, df_model.dim_model_key, df_date.dim_date_key)

In [0]:
display(df_fact)

## Writintg Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.fact_sales"):
    deltatable = DeltaTable.forName(spark, "cars_catalog.gold.fact_sales")

    deltatable.alias("target").merge(df_fact.alias("source"), 'target.dim_branch_key = source.dim_branch_key and target.dim_dealer_key = source.dim_dealer_key and target.dim_model_key = source.dim_model_key and target.dim_date_key = source.dim_date_key')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact.write.format('delta')\
        .mode("Overwrite")\
        .option('path', 'abfss://gold@sacarsdatalake.dfs.core.windows.net/fact_sales')\
        .saveAsTable("cars_catalog.gold.fact_sales")